# **Friend recommendation**

Related materials:

* https://snap.stanford.edu/data/ego-Facebook.html
* https://snap.stanford.edu/data/ego-Gplus.html
* https://snap.stanford.edu/data/ego-Twitter.html
* https://github.com/aditya-grover/node2vec
* https://github.com/pyg-team/pytorch_geometric
* https://arxiv.org/abs/1607.00653 -> https://arxiv.org/pdf/1607.00653.pdf
* https://arxiv.org/abs/1611.07308 -> https://arxiv.org/pdf/1611.07308.pdf


## Imports

In [2]:
!pip install fastapi==0.104.1 typing_extensions==4.8.0 gradio==3.41.0
import gradio as gr

import pandas as pd
import numpy as np

import networkx as nx

from bokeh.plotting import show
from bokeh.models import BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool, ColumnDataSource, MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool, PanTool, WheelZoomTool
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx


import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install pyg-lib -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

import requests
import pandas as pd
import io
import gzip
import random
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from torch_geometric.data import Data
import networkx as nx
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.nn.conv import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import os
from torch.optim import Adam
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn.models import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

!pip install wandb
import wandb

  Obtaining dependency information for fastapi==0.104.1 from https://files.pythonhosted.org/packages/f3/4f/0ce34195b63240b6693086496c9bab4ef23999112184399a3e88854c7674/fastapi-0.104.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions==4.8.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for gradio==3.41.0 from https://files.pythonhosted.org/packages/70/38/4fb11e6ddf8ecc77717389b553ea8dedca1a0574eee4fcbaf943e7dc34df/gradio-3.41.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<4.0.0,>=3.7.1 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4 from https://files.pythonhosted.org/packages/0a/2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


AttributeError: 'NoneType' object has no attribute 'replace'

## Data acquisition and analysis

In [ ]:
# Define the URL for the data
url = "https://snap.stanford.edu/data/facebook_combined.txt.gz"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content from the response
    content = response.content

    # Decompress the content using gzip
    with gzip.open(io.BytesIO(content), 'rt') as f:
        # Read the data into a Pandas DataFrame
        df = pd.read_csv(f, sep=" ", header=None, names=["source", "target"])

else:
    print("Failed to download the data. Status code:", response.status_code)

In [ ]:
def create_nx_graph_from_df(df):
  G = nx.Graph()
  nodes = np.unique(np.array(df.values).reshape(1,-1))
  edges = df.values
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  return G

In [ ]:
G = create_nx_graph_from_df(df)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot degree distribution
degree_centrality = nx.degree_centrality(G)
ax1.hist(list(degree_centrality.values()), bins=100, alpha=0.75)
ax1.set_title("Degree Centrality Distribution")
ax1.set_xlabel("Degree Centrality")
ax1.set_ylabel("Frequency")

# Plot clustering coefficient distribution
clustering_coefficients = nx.clustering(G).values()
ax2.hist(clustering_coefficients, bins=100, alpha=0.75)
ax2.set_title("Clustering Coefficient Distribution")
ax2.set_xlabel("Clustering Coefficient")
ax2.set_ylabel("Frequency")

plt.tight_layout()
plt.show()

## Data cleansing

In [ ]:
# Remove duplicate edges and self-edges
condition = df['source'] > df['target']
df.loc[condition, ['source', 'target']] = (df.loc[condition, ['target', 'source']].values)
df = df.drop_duplicates()
df = df[df['source'] != df['target']]

## Baseline Model

In [ ]:
g = nx.from_pandas_edgelist(df)

all_edges = g.edges()
train_edges = random.sample(g.edges(), k = int(0.8 * g.number_of_edges()))
test_edges = list(all_edges - train_edges)

all_negative_edges = list(nx.non_edges(g))
test_edges_false = random.sample(all_negative_edges, len(test_edges))

g_train = g.copy()
g_train.remove_edges_from(test_edges)

# Compute Preferential Attachment values from g_train
aa_matrix = np.zeros((g.number_of_nodes(), g.number_of_nodes()))
for u, v, p in nx.preferential_attachment(g_train): # (u, v) = node indices, p = Preferential Attachment value
  aa_matrix[u][v] = p
  aa_matrix[v][u] = p # make sure it's symmetric

# Normalize array
aa_matrix = aa_matrix / aa_matrix.max()

adj_sparse = nx.to_scipy_sparse_array(g)

def get_baseline_roc_score(edges_pos, edges_neg, score_matrix):
  # Store positive edge predictions, actual values
  preds_pos = []
  pos = []
  for edge in edges_pos:
    preds_pos.append(score_matrix[edge[0], edge[1]]) # predicted score
    pos.append(adj_sparse[edge[0], edge[1]]) # actual value (1 for positive)

  # Store negative edge predictions, actual values
  preds_neg = []
  neg = []
  for edge in edges_neg:
    preds_neg.append(score_matrix[edge[0], edge[1]]) # predicted score
    neg.append(adj_sparse[edge[0], edge[1]]) # actual value (0 for negative)

  # Calculate scores
  preds_all = np.hstack([preds_pos, preds_neg])
  labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
  roc_score = roc_auc_score(labels_all, preds_all)
  ap_score = average_precision_score(labels_all, preds_all)
  return roc_score, ap_score

# Calculate ROC AUC and Average Precision
aa_roc, aa_ap = get_baseline_roc_score(test_edges, test_edges_false, aa_matrix)

print('Baseline: Preferential Attachment ROC score: ', str(aa_roc))
print('Baseline: Preferential Attachment AP score: ', str(aa_ap))

<ipython-input-15-ade907476353>:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_edges = random.sample(g.edges(), k = int(0.8 * g.number_of_edges()))


Baseline: Preferential Attachment ROC score:  0.8267615825846976
Baseline: Preferential Attachment AP score:  0.8430094541217709


## Model

In [ ]:
class GCNEncoder(nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super(GCNEncoder, self).__init__()
    self.gcn_shared = GCNConv(in_channels, hidden_channels)
    self.gcn_mu = GCNConv(hidden_channels, out_channels)
    self.gcn_logvar = GCNConv(hidden_channels, out_channels)

  def forward(self, x, edge_index):
    x = F.relu(self.gcn_shared(x, edge_index))
    mu = self.gcn_mu(x, edge_index)
    logvar = self.gcn_logvar(x, edge_index)
    return mu, logvar


class DeepVGAE(VGAE):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super(DeepVGAE, self).__init__(encoder = GCNEncoder(in_channels, hidden_channels, out_channels), decoder = InnerProductDecoder())

  def forward(self, x, edge_index):
    z = self.encode(x, edge_index)
    adj_pred = self.decoder.forward_all(z)
    return adj_pred

  def loss(self, x, pos_edge_index, neg_edge_index):
    z = self.encode(x, pos_edge_index)

    pos_loss = -torch.log(self.decoder(z, pos_edge_index, sigmoid = True) + 1e-15).mean()
    neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid = True) + 1e-15).mean()

    kl_loss = 1 / x.size(0) * self.kl_loss()

    return pos_loss + neg_loss + kl_loss

  def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
    with torch.no_grad():
      z = self.encode(x, train_pos_edge_index)
    roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
    return roc_auc_score, average_precision_score

In [ ]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def create_input(config):
  # Hyperparameters
  learning_rate = config["learning_rate"]
  num_features = config["num_features"]
  num_epoch = config["num_epoch"]
  hidden_channels = config["hidden_channels"]
  out_channels = config["out_channels"]
  walk_length = config["walk_length"]
  context_size = config["context_size"]
  walks_per_node = config["walks_per_node"]

  edge_index = torch.tensor(df[['source', 'target']].values.T, dtype=torch.long)
  num_nodes = len(np.unique(np.array(df.values).reshape(1,-1)))
  train_n2v_model = Node2Vec(edge_index = edge_index, embedding_dim = num_features, walk_length = walk_length, context_size = context_size, walks_per_node = walks_per_node, sparse = True)
  x = train_n2v_model.forward()

  # split data before using node2vec
  data = Data(edge_index = edge_index, num_nodes = num_nodes)
  transform = RandomLinkSplit(is_undirected = True, neg_sampling_ratio = 1, num_test = 0.2, num_val = 0.1)
  train_data, val_data, test_data = transform(data)

  # use node2vec to create embeddings for train, test and val data seperately
  train_positive_edges_mask = (train_data.edge_label == 1)
  train_positive_edges = train_data.edge_index[:, train_positive_edges_mask]
  train_data = Data(x = x, edge_index = train_positive_edges)
  train_neg_edge_index = negative_sampling(train_data.edge_index, num_nodes = train_data.num_nodes, num_neg_samples = train_positive_edges.size(1))
  train_data.pos_edge_index = train_data.edge_index
  train_data.neg_edge_index = train_neg_edge_index
  train_data.edge_index = torch.cat([train_data.pos_edge_index, train_data.neg_edge_index], dim=1)


  test_positive_edges_mask = (test_data.edge_label == 1)
  test_positive_edges = test_data.edge_label_index[:, test_positive_edges_mask]
  test_data = Data(x = x, edge_index = test_positive_edges)
  test_neg_edge_index = negative_sampling(test_data.edge_index, num_nodes = test_data.num_nodes, num_neg_samples = test_positive_edges.size(1))
  test_data.pos_edge_index = test_data.edge_index
  test_data.neg_edge_index = test_neg_edge_index
  test_data.edge_index = torch.cat([test_data.pos_edge_index, test_data.neg_edge_index], dim=1)


  val_positive_edges_mask = (val_data.edge_label == 1)
  val_positive_edges = val_data.edge_label_index[:, val_positive_edges_mask]
  val_data = Data(x = x, edge_index = val_positive_edges)
  val_neg_edge_index = negative_sampling(val_data.edge_index, num_nodes = val_data.num_nodes, num_neg_samples = val_positive_edges.size(1))
  val_data.pos_edge_index = val_data.edge_index
  val_data.neg_edge_index = val_neg_edge_index
  val_data.edge_index = torch.cat([val_data.pos_edge_index, val_data.neg_edge_index], dim=1)

  return train_data, test_data, val_data

def train_model(config, train_data, test_data, val_data, log_wandb):
  learning_rate = config["learning_rate"]
  num_features = config["num_features"]
  num_epoch = config["num_epoch"]
  hidden_channels = config["hidden_channels"]
  out_channels = config["out_channels"]
  walk_length = config["walk_length"]
  context_size = config["context_size"]
  walks_per_node = config["walks_per_node"]

  model = DeepVGAE(num_features, hidden_channels, out_channels).to(device)
  optimizer = Adam(model.parameters(), lr = learning_rate)

  for epoch in range(num_epoch):
    model.train()
    optimizer.zero_grad()
    val_loss = model.loss(val_data.x, val_data.pos_edge_index, val_data.neg_edge_index)
    train_loss = model.loss(train_data.x, train_data.pos_edge_index, train_data.neg_edge_index)
    train_loss.backward()
    optimizer.step()
    model.eval()
    roc_auc, ap = model.single_test(train_data.x, train_data.pos_edge_index, test_data.pos_edge_index, test_data.neg_edge_index)

    if(log_wandb == True):
      wandb.log({"Training Loss" : train_loss, "Validation Loss" : val_loss, "ROC / AUC" : roc_auc, "Precision" : ap})

  return model, roc_auc, ap

def wandb_train_model():
  wandb.init()

  train_data, test_data, val_data = create_input(wandb.config)

  train_data = train_data.to(device)
  test_data = test_data.to(device)
  val_data = val_data.to(device)

  model, roc_auc, ap = train_model(wandb.config, train_data, test_data, val_data, True)

##  Training the model

In [ ]:
# Hyperparameter optimalization
sweep_configuration = {
  "method": "bayes",
  "name": "sweep",
  "metric": {"goal": "minimize", "name": "Validation Loss"},
  "parameters": {
    "learning_rate" : {"max" : 0.1, "min" : 0.0001},
    "num_features" : {"min" : 2, "max" : 128},
    "num_epoch" : {"min" : 25, "max" : 200},
    "hidden_channels" : {"min" : 1, "max" : 64},
    "out_channels" : {"min" : 1, "max" : 16},
    "walk_length" : {"min" : 10, "max" : 100},
    "context_size" : {"min" : 1, "max" : 10},
    "walks_per_node" : {"min" : 1, "max" : 10},
  },
}

#wandb.login()
os.environ["WANDB_API_KEY"] = "858ab781cf5ec123e162cc8cfe304c1a096f76cb"

project = "Deep Learning Homework"
sweep_id = wandb.sweep(sweep = sweep_configuration, project = project)
count = 1
wandb.agent(sweep_id, function = wandb_train_model, count = count)

sweep = wandb.Api().sweep(f'{os.getenv("WANDB_ENTITY")}/{project}/{sweep_id}')
best_run = sweep.best_run()
best_parameters = best_run.config

wandb.finish()

# Training a model with the best parameters
train_data, test_data, val_data = create_input(best_parameters)
train_data = train_data.to(device)
test_data = test_data.to(device)
val_data = val_data.to(device)
model, roc_auc, ap = train_model(best_parameters, train_data, test_data, val_data, False)

Create sweep with ID: vd2ci63h
Sweep URL: https://wandb.ai/szabolcsrikli/Deep%20Learning%20Homework/sweeps/vd2ci63h


wandb: Agent Starting Run: v0tbanx9 with config:
wandb: 	context_size: 5
wandb: 	hidden_channels: 28
wandb: 	learning_rate: 0.072455359207828
wandb: 	num_epoch: 185
wandb: 	num_features: 103
wandb: 	out_channels: 7
wandb: 	walk_length: 53
wandb: 	walks_per_node: 6


Precision,▂▁▁▃▅▅▆▆▆▇▇▇▇▇▇█████████████████████████
ROC / AUC,▂▁▃▄▅▅▆▆▇▇▇▇▇▇▇█████████████████████████
Training Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▂▂▃▃▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Precision,0.95272
ROC / AUC,0.95733
Training Loss,0.84915
Validation Loss,1.79579


wandb: Sorting runs by +summary_metrics.Validation Loss


In [ ]:
print(roc_auc, ap)

0.9583468248300421 0.9539381822788823


## Creating and cleansing output

In [ ]:
# Predicting adjacency matrix containing edge probabilities
final_edge_index = model.forward(test_data.x, test_data.edge_index)

# Creating a pandas dataframe from the adjacency matrix
pred_adj_mat = final_edge_index
pred_adj_mat_numpy = pred_adj_mat.cpu().detach().numpy()
G_pred = nx.from_numpy_array(pred_adj_mat_numpy)
df_pred = nx.to_pandas_edgelist(G_pred)

# Deleting duplicate edges and self-edges
condition = df_pred['source'] > df_pred['target']
df_pred.loc[condition, ['source', 'target']] = (df_pred.loc[condition, ['target', 'source']].values)
df_pred = df_pred.drop_duplicates()
df_pred = df_pred[df_pred['source'] != df_pred['target']]

In [ ]:
df_pred['weight'].hist(bins=100)

In [ ]:
def update(node_name):
  index = int(node_name)

  # Retrieving edges of the node with name: index
  condition = df_pred['source'] == index
  predicted_edges = df_pred.loc[condition, df_pred.columns]
  condition = df_pred['target'] == index
  predicted_edges = pd.concat([predicted_edges, df_pred.loc[condition, df_pred.columns]])

  # Sorting edges by probability
  predicted_edges = predicted_edges.sort_values(by = ['weight'], ascending = False)

  # Plot
  fig = plt.figure()
  ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
  ax.hist(x = predicted_edges['weight'], bins = 100, density = True, range = [0.0, 1.0])
  plt.grid(color = 'grey', linestyle='--', linewidth=0.5)
  plt.title("Probability density of edges")
  plt.xlabel("x")
  plt.ylabel("Density")

  return predicted_edges.head(10), fig;

# Displaying the styled dataframe in Gradio
with gr.Blocks() as demo:
  with gr.Row():
    inp = gr.Textbox(placeholder = "Which node's recommendations do you want to see?")
    out = gr.DataFrame()
    out2 = gr.Plot()
  btn = gr.Button("Run")
  btn.click(fn = update, inputs = [inp], outputs = [out, out2])

if __name__ == "__main__":
  demo.launch(show_api = False, share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

model.eval()
with torch.no_grad():
  all_pos_edge_index = torch.cat([train_data.pos_edge_index, val_data.pos_edge_index], dim=1)
  all_neg_edge_index = torch.cat([train_data.neg_edge_index, val_data.neg_edge_index], dim=1)

all_pos_df = pd.DataFrame()
all_pos_df['source'] = all_pos_edge_index[0]
all_pos_df['target'] = all_pos_edge_index[1]
all_pos_df['edge'] = 1

all_neg_df = pd.DataFrame()
all_neg_df['source'] = all_neg_edge_index[0]
all_neg_df['target'] = all_neg_edge_index[1]
all_neg_df['edge'] = 0

df_all_edges = pd.concat([all_pos_df, all_neg_df], ignore_index=True)

condition = df_all_edges['source'] > df_all_edges['target']
df_all_edges.loc[condition, ['source', 'target']] = (df_all_edges.loc[condition, ['target', 'source']].values)
df_all_edges = df_all_edges.drop_duplicates()
df_all_edges = df_all_edges[df_all_edges['source'] != df_all_edges['target']]

df_pred['source'] = df_pred['source'].astype(int)
df_pred['target'] = df_pred['target'].astype(int)

merged_df = pd.merge(df_all_edges, df_pred, on=['source', 'target'], how='left', indicator=True)
# Create a new column based on the merge results
merged_df['both'] = merged_df['_merge'] == 'both'
merged_df.drop('_merge', axis=1, inplace=True)
merged_df = merged_df.loc[merged_df['both']==True]

pred = merged_df['weight']
all_labels = merged_df['edge']
fpr, tpr, _ = roc_curve(all_labels, pred)

roc_auc = roc_auc_score(all_labels, pred)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()